## OSWorld

In [ ]:
from agential.agents.OSWorldBaseline.agent import OSWorldBaselineAgent

In [8]:
"""
To get accessibility_tree, retreive obs by running this line
- obs, reward, done, info = env.step("pyautogui.rightClick()")
- obs["accessibility_tree"]
"""
with open("accessibility_tree.txt", "r", encoding="utf-8") as file:
    accessibility_tree = file.read()

instruction = "Please help me to find the nearest restaurant."

"""
Image of screen is to be provided by user
"""
obs = {"screenshot": open("output_image.jpeg", 'rb').read()}


In [4]:
agent = OSWorldBaselineAgent(
    model="gpt-4o",
    observation_type="screenshot",
)

In [5]:
response, actions, messages = agent.generate(instruction, obs)

In [ ]:
print(f"Response: {response}")
print(f"Actions: {actions}")
print(f"Messages: {messages}")

In [57]:
import re
input_string = "Here is some text; ```python\ncode snippet here\nWAIT```"

In [62]:
input_string = "\n".join([line.strip() for line in input_string.split(';') if line.strip()])
if input_string.strip() in ['WAIT', 'DONE', 'FAIL']:
    print([input_string.strip()])

pattern = r"```(?:\w+\s+)?(.*?)```"
matches = re.findall(pattern, input_string, re.DOTALL)

codes = []


In [59]:
matches

['code snippet here\nWAIT']

In [63]:
for match in matches:
    match = match.strip()
    commands = ['WAIT', 'DONE', 'FAIL']  # fixme: updates this part when we have more commands

    if match in commands:
        codes.append(match.strip())
    elif match.split('\n')[-1] in commands:
        if len(match.split('\n')) > 1:
            codes.append("\n".join(match.split('\n')[:-1]))
            print(f"1: {codes}")
        codes.append(match.split('\n')[-1])
        print(f"2: {codes}")
    else:
        codes.append(match)

1: ['code snippet here']
2: ['code snippet here', 'WAIT']


In [64]:
print("\n".join(match.split('\n')[:-1]))

code snippet here


In [55]:
print(input_string.strip())

WAIT
DONE
FAIL


In [ ]:
matches = re.findall(pattern, input_string, re.DOTALL)